In [24]:
import os
from PIL import Image
import torch
from torchvision import models, transforms
from annoy import AnnoyIndex

# Path to your image folder
images_folder = r'C:\Users\shiva\archive\PetImages\Cat'

# List the images in the folder
images = os.listdir(images_folder)

# Load the pre-trained ResNet18 model
weights = models.ResNet18_Weights.IMAGENET1K_V1
model = models.resnet18(weights=weights)

# Set the model to evaluation mode
model.eval()

# Remove the final fully connected layer to get a feature vector of 512 dimensions
# This is the model without the final classification layer
model = torch.nn.Sequential(*list(model.children())[:-1])

# Define the transformation pipeline
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Initialize the Annoy index with 512 dimensions (as we now want a 512-dimensional vector)
annoy_index = AnnoyIndex(512, 'angular')

# Process each image in the folder
for i in range(len(images)):
    image = Image.open(os.path.join(images_folder, images[i]))
    input_tensor = transform(image).unsqueeze(0)  # Add batch dimension

    # Ensure the image has 3 color channels
    if input_tensor.size()[1] == 3:
        with torch.no_grad():
            # Pass through the modified ResNet18 to get the feature vector
            features = model(input_tensor)  # The output is 512-dimensional

            # Flatten the features to 1D vector
            feature_vector = features.view(-1).numpy()  # Convert to numpy array for Annoy

        # Add the feature vector to the Annoy index
        annoy_index.add_item(i, feature_vector)

        # Progress feedback
        if i % 100 == 0:
            print(f'Processed {i} images.')

# Build the index and save it
annoy_index.build(10)
annoy_index.save('cat_index.ann')

print("Index built and saved successfully!")


Processed 0 images.
Processed 100 images.
Processed 200 images.
Processed 300 images.
Processed 400 images.
Processed 500 images.
Processed 600 images.
Processed 700 images.
Processed 800 images.
Processed 900 images.
Processed 1000 images.
Processed 1100 images.
Processed 1200 images.
Processed 1300 images.
Processed 1400 images.
Processed 1500 images.
Processed 1600 images.
Processed 1700 images.
Processed 1800 images.
Processed 1900 images.
Processed 2000 images.
Processed 2100 images.
Processed 2200 images.
Processed 2300 images.
Processed 2400 images.
Processed 2500 images.
Processed 2600 images.
Processed 2700 images.
Processed 2800 images.
Processed 2900 images.
Processed 3000 images.
Processed 3100 images.
Processed 3200 images.
Processed 3300 images.
Processed 3400 images.
Processed 3500 images.
Processed 3600 images.
Processed 3700 images.
Processed 3800 images.
Processed 3900 images.
Processed 4000 images.
Processed 4100 images.
Processed 4200 images.
Processed 4300 images.
